In [ ]:
import torch
import os
from torch import nn, optim
from torch.utils.data import DataLoader
from gcngru import GRU_Model
from training_data import TrainingData

def load_processed_data(file_path):
    return np.load(file_path, allow_pickle=True)

def prepare_data(data):
    X = TrainingData.trainingData_iterator(data)
    return X

def train(model, dataloader, criterion, optimizer, device):
    model.train()
    for signal_processed, Adj_dist_matrix, bdi_label in dataloader:
        signal_processed = signal_processed.to(device).float()
        Adj_dist_matrix = Adj_dist_matrix.to(device).float()
        bdi_label = bdi_label.to(device).float()

        optimizer.zero_grad()
        output, _ = model(signal_processed, Adj_dist_matrix)
        loss = criterion(output, bdi_label)
        loss.backward()
        optimizer.step()

    print(f'Training Loss: {loss.item()}')

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for signal_processed, Adj_dist_matrix, bdi_label in dataloader:
            signal_processed = signal_processed.to(device).float()
            Adj_dist_matrix = Adj_dist_matrix.to(device).float()
            bdi_label = bdi_label.to(device).float()

            output, _ = model(signal_processed, Adj_dist_matrix)
            loss = criterion(output, bdi_label)
            total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Evaluation Loss: {avg_loss}')

def main():
    # Load processed signals
    processed_signals_path = 'processed_signals.npy'
    training_data = load_processed_data(processed_signals_path)

    # Prepare data for training
    data = prepare_data(training_data)
    dataset = EEGDataset(data)

    # Dataset and DataLoader
    batch_size = 40
    train_test_split_ratio = 0.7
    train_dataloader, test_dataloader = train_test_datasets(dataset, batch_size, train_test_split_ratio)

    # Model, loss function, optimizer
    nodes_dim = 16
    node_features_dim = 16
    hidden_dim = 64
    num_layers = 2
    dropout_prob = 0.5
    seq_len = 10  # Example sequence length, adjust as needed

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GRU_Model(nodes_dim, node_features_dim, hidden_dim, num_layers, dropout_prob, seq_len).to(device)
    criterion = nn.MSELoss()  # Change this to the appropriate loss for your task
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 10
    for epoch in range(num_epochs):
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        train(model, train_dataloader, criterion, optimizer, device)
        evaluate(model, test_dataloader, criterion, device)
        print(f'Epoch [{epoch+1}/{num_epochs}] completed.')

if __name__ == "__main__":
    main()
